In [3]:
from pathlib import Path
import sys

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd
import librosa
from scipy.io import wavfile

from Code.audio import AudioPreproc, PreprocCfg

run = 1
dataset = 4

input_dir_base = PROJECT_ROOT / "Database" / "data" / "audio"
output_dir_base = PROJECT_ROOT / "Database" / "tmp" / f"audioPreprocTry{run:02d}"
output_dir_base.mkdir(parents=True, exist_ok=True)

labels = ["contar", "proporcion", "salir"]

config = PreprocCfg(
    target_sr=16000, t_sec=1.2,
    frame_ms=25.0, hop_ms=10.0,
    highpass_hz=40.0, hp_order=2,
    preemph_a=0.97,
    vad_thresh_db=-35.0, vad_win_ms=20.0,
    vad_min_ms=120.0, vad_expand_ms=60.0,
    norm_mode="rms", rms_target_dbfs=-20.0,
    peak_ref=0.98, max_gain_db=18.0,
    gate_dbfs=-60.0, pad_mode="edge",
)
pre = AudioPreproc(config)

y_proc, sr_proc, file_names, class_names = [], [], [], []

for cls in labels:
    input_dir = input_dir_base / cls
    output_dir = output_dir_base / cls
    output_dir.mkdir(parents=True, exist_ok=True)

    for audio_path in sorted(input_dir.glob("*.wav")):
        y, sr = pre.preprocesar_desde_path(audio_path)

        # guardar WAV procesado (float32 -> int16)
        y_int16 = np.clip(y * 32767.0, -32768, 32767).astype(np.int16)
        out_path = output_dir / audio_path.name
        wavfile.write(out_path, sr, y_int16)

        y_proc.append(y)
        sr_proc.append(sr)
        file_names.append(audio_path.name)
        class_names.append(cls)

# Tabla de estadísticos por archivo
filas = []
for cls, fname, y, sr in zip(class_names, file_names, y_proc, sr_proc):
    filas.append({
        "Clase": cls,
        "Archivo": fname,
        "Duración (s)": len(y) / sr,
        "Mín": float(np.min(y)),
        "Máx": float(np.max(y)),
        "Energía RMS": float(np.sqrt(np.mean(y**2))),
    })

df = pd.DataFrame(filas)
df.to_csv(output_dir_base / "estadisticos.csv", index=False)
df  # muestra la tabla en el notebook


,Clase,Archivo,Duración (s),Mín,Máx,Energía RMS
0,contar,Contar001.wav,1.2,-0.141450,0.143980,0.085808
1,contar,Contar002.wav,1.2,-0.148299,0.181640,0.065979
2,contar,Contar003.wav,1.2,-0.148536,0.143036,0.101841
3,contar,Contar004.wav,1.2,-0.148297,0.142823,0.065664
4,contar,Contar005.wav,1.2,-0.148004,0.141549,0.102665
5,contar,Contar006.wav,1.2,-0.141642,0.146344,0.070738
6,contar,Contar007.wav,1.2,-0.148731,0.141695,0.118796
7,contar,Contar008.wav,1.2,-0.143153,0.148227,0.105722
8,proporcion,Proporcion001.wav,1.2,-0.146279,0.197177,0.096634
9,proporcion,Proporcion002.wav,1.2,-0.161453,0.172382,0.068324
